In [1]:
import pandas as pd
import time

In [2]:
with open('onegin.txt', 'r') as f:
    d = f.read()

In [3]:
oneg = d.split('\n')

In [4]:
a = []
b = []
count = 0
for i in oneg:
    if i[:2] == '\t\t' and i[:4] != '\t\t……':
        a.append(i[2:])
    count += 1
    if count % 4 == 0 and a != []:
        b.append(' '.join(a))
        a = []

In [5]:
text = [x.lower() for x in b]

In [6]:
text[:10]

['he мысля гордый свет забавить, вниманье дружбы возлюбя, хотел бы я тебе представить',
 'залог достойнее тебя, достойнее души прекрасной, святой исполненной мечты, поэзии живой и ясной,',
 'высоких дум и простоты; но так и быть – рукой пристрастной прими собранье пёстрых глав, полусмешных, полупечальных,',
 'простонародных, идеальных, небрежный плод моих забав, бессонниц, лёгких вдохновений, незрелых и увядших лет,',
 'ума холодных наблюдений и сердца горестных замет.',
 '«мой дядя самых честных правил, когда не в шутку занемог, он уважать себя заставил и лучше выдумать не мог.',
 'его пример другим наука; но, боже мой, какая скука с больным сидеть и день и ночь, не отходя ни шагу прочь!',
 'какое низкое коварство полуживого забавлять, ему подушки поправлять, печально подносить лекарство,',
 'вздыхать и думать про себя: когда же чёрт возьмёт тебя!»',
 'так думал молодой повеса, летя в пыли на почтовых,']

In [7]:
CHARS = set('абвгдеёжзийклмнопрстуфхцчшщыьъэюя.,?! ')

In [8]:
len(CHARS)

38

In [9]:
text2 = []
for i in text:
    line = ''
    for j in i:
        if j in CHARS:
            line = line+j 
    text2.append(line)

In [10]:
text2[:10]

[' мысля гордый свет забавить, вниманье дружбы возлюбя, хотел бы я тебе представить',
 'залог достойнее тебя, достойнее души прекрасной, святой исполненной мечты, поэзии живой и ясной,',
 'высоких дум и простоты но так и быть  рукой пристрастной прими собранье пёстрых глав, полусмешных, полупечальных,',
 'простонародных, идеальных, небрежный плод моих забав, бессонниц, лёгких вдохновений, незрелых и увядших лет,',
 'ума холодных наблюдений и сердца горестных замет.',
 'мой дядя самых честных правил, когда не в шутку занемог, он уважать себя заставил и лучше выдумать не мог.',
 'его пример другим наука но, боже мой, какая скука с больным сидеть и день и ночь, не отходя ни шагу прочь!',
 'какое низкое коварство полуживого забавлять, ему подушки поправлять, печально подносить лекарство,',
 'вздыхать и думать про себя когда же чёрт возьмёт тебя!',
 'так думал молодой повеса, летя в пыли на почтовых,']

In [11]:
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]

In [12]:
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [13]:
CHAR_TO_INDEX

{'none': 0,
 'ъ': 1,
 ' ': 2,
 'з': 3,
 'щ': 4,
 '!': 5,
 'к': 6,
 'д': 7,
 'ц': 8,
 'й': 9,
 'ь': 10,
 'и': 11,
 'ф': 12,
 'х': 13,
 ',': 14,
 'л': 15,
 'а': 16,
 'с': 17,
 'ё': 18,
 'б': 19,
 'м': 20,
 'р': 21,
 'о': 22,
 '?': 23,
 'т': 24,
 'ш': 25,
 'ы': 26,
 'е': 27,
 'ч': 28,
 'я': 29,
 '.': 30,
 'в': 31,
 'э': 32,
 'ю': 33,
 'ж': 34,
 'г': 35,
 'п': 36,
 'у': 37,
 'н': 38}

In [14]:
import torch

In [15]:
MAX_LEN = 50

In [16]:
X = torch.zeros((len(text2), MAX_LEN), dtype=int)

In [17]:
for i in range(len(text2)):
    for j, w in enumerate(text2[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [18]:
X[0:1]

tensor([[ 2, 20, 26, 17, 15, 29,  2, 35, 22, 21,  7, 26,  9,  2, 17, 31, 27, 24,
          2,  3, 16, 19, 16, 31, 11, 24, 10, 14,  2, 31, 38, 11, 20, 16, 38, 10,
         27,  2,  7, 21, 37, 34, 19, 26,  2, 31, 22,  3, 15, 33]])

In [19]:
class Network(torch.nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
        self.gru = torch.nn.RNN(28, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [20]:
model = Network()

In [21]:
model.forward(X[0:1])[0].shape

torch.Size([1, 50, 39])

In [22]:
dev = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [23]:
model = model.to(dev)

In [24]:
def generate_sentence():
    sentence = ['м', 'о', 'й', ' ', 'д', 'я', 'д', 'я']
    state = None
    for i in range(MAX_LEN):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long).to(dev)
        if i == 0:
            result, state = model.forward(X)
        else:
            result, state = model.forward_state(X, state)
        prediction = result[0, -1, :]
        index_of_prediction = prediction.argmax()
        if i >= len(sentence) - 1:
            if index_of_prediction == 0:
                break
        sentence.append(INDEX_TO_CHAR[index_of_prediction])
    print(''.join(sentence))

In [25]:
generate_sentence()

мой дядяыыхыыфафзццкццц!цыт ыыыткхцыыззцзлкрыцыыйы?й!ыаццз


In [26]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.005, momentum=0.7)

In [27]:
for ep in range(10000):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()
        X_batch = X_batch.to(dev)
        Y_batch = Y_batch.to(dev)

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    
    if ep % 100 == 0:
        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        generate_sentence()


Epoch 0. Time: 0.052, Train loss: 3.663
мой дядяыыхыыфафзццкццц!цыт ыыыткхцыыззцзлкрыцыыйыой!ыаццз
Epoch 100. Time: 0.043, Train loss: 2.663
мой дядя   со   нооеенооон        ноооооооон        нооооо
Epoch 200. Time: 0.060, Train loss: 2.533
мой дядя т по   носолпоролт        поорррррррраа        по
Epoch 300. Time: 0.029, Train loss: 2.463
мой дядя с по   птпорпоррорнаснаат        порррррррраааа  
Epoch 400. Time: 0.027, Train loss: 2.414
мой дядя с по   птпррпоррораасноан   т    соропорртрасонаа
Epoch 500. Time: 0.070, Train loss: 2.374
мой дядяас па    тполпоррррноо оаааи  пр    нороппрросасоо


KeyboardInterrupt: 

In [ ]:
class GRU_Network(torch.nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
        self.gru = torch.nn.GRU(28, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [ ]:
model = Network()

In [ ]:
model = model.to(dev)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=.001)

In [ ]:
def train_net(epocha):
    for ep in range(epocha):
        start = time.time()
        train_loss = 0.
        train_passed = 0

        for i in range(int(len(X) / 100)):
            batch = X[i * 100:(i + 1) * 100]
            X_batch = batch[:, :-1]
            Y_batch = batch[:, 1:].flatten()
            X_batch = X_batch.to(dev)
            Y_batch = Y_batch.to(dev)

            optimizer.zero_grad()
            answers, _ = model.forward(X_batch)
            answers = answers.view(-1, len(INDEX_TO_CHAR))
            loss = criterion(answers, Y_batch)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()
            train_passed += 1


        if ep % 100 == 0:
            print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
            generate_sentence()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=.001)

In [ ]:
train_net(5000)

In [ ]:
# лосс больше не падает - сеть генерирует отдельные слова. Попробовал ванильную RNN и GRU. Результат сильно не улушился.